In [1]:
# Load dependencies

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.models as models
import torch.optim as optim
import os
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchattacks

In [2]:
# Setup CUDA Device

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
model_name = "Mobilenetv3Small"
version = "v1"
training_name = "ARD_NotPretrainMobilenet"

height = 224
num_classes = 7
epochs = 100
lr = 0.0001
lr_factor = 0.1
lr_threshold = 6
weight_decay = 0.0002
batch_size = 32

# Attack hyperparameters 
epsilon = 8.0 / 255
alpha = 2.0 / 255
steps = 10

# Knowledge Distillation hyperparameters
temp = 20.0
alpha = 1.0

In [4]:
# Graph writer initialize for data visualization

writer = SummaryWriter("runs/trashbox/" + f'{training_name}--{model_name}.{version}')

In [5]:

# Initialize pre-trained weights - IMAGENET
# Get the preprocessing for the respective pre-trained model

preprocessing = transforms.Compose([
    transforms.RandomResizedCrop((height, height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=mean, std=std)
])
test_preprocessing = transforms.Compose([
    transforms.RandomResizedCrop((height, height)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=mean, std=std)
])

In [6]:
# Training and Validation dataset

print('===>> Preparing data...')
trash_train_dataset = torchvision.datasets.ImageFolder('dataset/trashbox/train', transform=preprocessing)
trash_train_loader = torch.utils.data.DataLoader(dataset=trash_train_dataset, shuffle=True, batch_size=batch_size)
trash_val_dataset = torchvision.datasets.ImageFolder('dataset/trashbox/val', transform=test_preprocessing)
trash_val_loader = torch.utils.data.DataLoader(dataset=trash_val_dataset, shuffle=True, batch_size=batch_size)

===>> Preparing data...


In [7]:
# Setup model 

print('====>> Setting up teacher model...')
checkpoint = torch.load('./best_trained_models/best_AT--Googlenet.v1_epoch98.pth')
# model = local_models.xception(num_classes=num_classes).to(device)
teacher_model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
infeatures = teacher_model.fc.in_features

for param in teacher_model.parameters():
    param.requires_grad = False



teacher_model.fc = nn.Linear(infeatures, num_classes, True)



if 'module' in list(checkpoint['net'].keys())[0]:
    new_state_dict = {k.replace("module.", ""): v for k, v in checkpoint['net'].items()}
    teacher_model.load_state_dict(new_state_dict)
else:
    teacher_model.load_state_dict(checkpoint['net'])

teacher_model = teacher_model.to(device)


====>> Setting up teacher model...


FileNotFoundError: [Errno 2] No such file or directory: './best_trained_models/best_AT--Googlenet.v1_epoch98.pth'

In [ ]:
# Setup model 

print('====>> Setting up student model...')
# model = local_models.xception(num_classes=num_classes).to(device)
model = models.mobilenet_v3_small(weights=None)
model.classifier = nn.Sequential(
    nn.Linear(in_features=576, out_features=1024, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1024, out_features=num_classes, bias=True)
)
model = model.to(device)


====>> Setting up student model...


In [ ]:
# Setup Adversarial attack 

attack = torchattacks.PGD(model=model, eps=epsilon, alpha=alpha, steps=steps)

class AttackPGD(nn.Module):
    def __init__(self, basic_net, config):
        super(AttackPGD, self).__init__()
        self.basic_net = basic_net
        self.step_size = config['step_size']
        self.epsilon = config['epsilon']
        self.num_steps = config['num_steps']

    def forward(self, inputs, targets):
        x = inputs.detach()
        x = x + torch.zeros_like(x).uniform_(-self.epsilon, self.epsilon)
        for i in range(self.num_steps):
            x.requires_grad_()
            with torch.enable_grad():
                loss = F.cross_entropy(self.basic_net(x), targets, size_average=False)
            grad = torch.autograd.grad(loss, [x])[0]
            x = x.detach() + self.step_size*torch.sign(grad.detach())
            x = torch.min(torch.max(x, inputs - self.epsilon), inputs + self.epsilon)
            x = torch.clamp(x, 0.0, 1.0)
        return self.basic_net(x), x

In [ ]:
config = {
    'step_size' : alpha,
    'epsilon' : epsilon, 
    'num_steps' : steps
}
net = AttackPGD(model, config)

In [ ]:
if device == 'cuda':
    cudnn.benchmark = True

In [ ]:
# Setup Loss function

XENT_loss = nn.CrossEntropyLoss()
KL_loss = nn.KLDivLoss()

In [ ]:
# Train Loop

def train(epoch, optimizer):
    train_loss = 0
    correct = 0
    total = 0
    adv_correct = 0
    net.train()
    iterator = tqdm(trash_train_loader, ncols=0, leave=False)
    for i, (inputs, targets)in enumerate(iterator):
        inputs, targets = inputs.to(device),targets.to(device)

        optimizer.zero_grad()
        # adv_image = attack(inputs, targets)
        output,per_inputs = net(inputs, targets)
        teacher_output = teacher_model(inputs)
        basic_output = model(inputs)
        # adv_basic_output = model(adv_image)
        # _, predicted = adv_basic_output.max(1)
        
        loss =  alpha* temp* temp*KL_loss(F.log_softmax(output/ temp, dim=1),F.softmax(teacher_output/ temp, dim=1))+(1.0- alpha)*XENT_loss(basic_output, targets)
        loss.backward()
    
        optimizer.step()
        train_loss += loss.item()
        iterator.set_description(str(loss.item()))       
        total += targets.size(0)
        
        _, predicted = basic_output.max(1)
        correct += predicted.eq(targets).sum().item()
        _, adv_predicted = output.max(1)
        adv_correct += adv_predicted.eq(targets).sum().item()
        
    validation_loss = train_loss  / len(trash_val_dataset)
    train_adv_accuracy = 100.0 * correct / total
    adv_train_adv_accuracy = 100.0 * adv_correct / total
    
    print('\nTotal adversarial train accuarcy:', 100. * correct / total)
    print('Total adversarial train loss:', train_loss)
    
    writer.add_scalar('Train loss: ' + model_name, validation_loss, epoch)
    writer.add_scalar('Train accuracy: ' + model_name, train_adv_accuracy, epoch)
    writer.add_scalar('Adversarial Train accuracy: ' + model_name, adv_train_adv_accuracy, epoch)

In [ ]:
# Test function
best_loss = float(0)

def test(epoch, optimizer):
    global best_loss
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    with torch.no_grad():
        iterator = tqdm(trash_val_loader, ncols=0, leave=False)
        for i, (inputs, targets) in enumerate(iterator):
            inputs, targets = inputs.to(device), targets.to(device)
            total += targets.size(0)
        
            adv_outputs, pert_inputs = net(inputs, targets)
            natural_outputs = model(inputs)
            _, adv_predicted = adv_outputs.max(1)
            _, natural_predicted = natural_outputs.max(1)
            
            benign_correct += natural_predicted.eq(targets).sum().item()
            adv_correct += adv_predicted.eq(targets).sum().item()

            iterator.set_description(str(adv_predicted.eq(targets).sum().item()/targets.size(0)))
    
    benign_val_accuracy = 100.0 * benign_correct / total
    adv_val_accuracy = 100.0 * adv_correct / total 
    
    print('\nTotal benign test accuarcy:', benign_val_accuracy)
    print('Total adversarial test Accuarcy:', adv_val_accuracy)
    
    # Graph
    writer.add_scalar("Natural test accuracy: " + model_name, benign_val_accuracy, epoch)
    writer.add_scalar("Adversarial test accuracy: " + model_name, adv_val_accuracy, epoch)
    
    # Save checkpoint
    state = {
        'epoch' : epoch,
        'net': model.state_dict(),
        'optim' : optimizer.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + f'{training_name}--{model_name}.{version}.pth')
    if benign_val_accuracy > best_loss:
        print(f'Model saved: f{benign_val_accuracy}')
        torch.save(state, './trained_model/' + f'best_{training_name}_{model_name}_{version}_epoch{epoch}.pth')
    print('Model Saved!')
    return benign_val_accuracy, adv_val_accuracy

In [ ]:
def main():
    learning_rate = lr
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0002)
    model_path = f'./checkpoint/{training_name}--{model_name}.{version}.pth'
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=lr_threshold, factor=lr_factor)
    if os.path.exists(model_path):
        # Load the saved model and optimizer state
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['net'])
        optimizer.load_state_dict(checkpoint['optim'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"=> Loaded checkpoint '{model_path}' (epoch {start_epoch})")
    else:
        start_epoch = 0
        print(f"=> No checkpoint found at '{model_path}'. Starting training from scratch.")


    for epoch in range(start_epoch, epochs):
        train_loss = train(epoch, optimizer)
        benign_val_accuracy , _ =test(epoch, optimizer)
        scheduler.step(metrics=benign_val_accuracy, epoch=epoch)
        scheduler.print_lr(True, model.parameters(), learning_rate, epoch)

In [ ]:
if __name__ == '__main__':
    main()

=> No checkpoint found at './checkpoint/ARD_NotPretrainMobilenet--Mobilenetv3Small.v1.pth'. Starting training from scratch.


  0% 0/447 [00:00<?, ?it/s]c:\Users\MY PC\anaconda3\envs\thesisaimodel\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
c:\Users\MY PC\anaconda3\envs\thesisaimodel\Lib\site-packages\torch\nn\functional.py:2949: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0.1980627477169037:   4% 17/447 [00:10<04:46,  1.50it/s] c:\Users\MY PC\anaconda3\envs\thesisaimodel\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Total adversarial train accuarcy: 21.556131381749424
Total adversarial train loss: 80.88373227417469

[ Test epoch: 0 ]


c:\Users\MY PC\anaconda3\envs\thesisaimodel\Lib\site-packages\torch\optim\lr_scheduler.py:1056: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Total benign test accuarcy: 28.017967434025827
Total adversarial test Accuarcy: 17.85513756316676
Model Saved!
Epoch 00000: adjusting learning rate of group <generator object Module.parameters at 0x00000224F86A7680> to 1.0000e-04.


...


Total adversarial train accuarcy: 50.262623433013516
Total adversarial train loss: 23.278761718422174

[ Test epoch: 46 ]



Total benign test accuarcy: 51.60022459292532
Total adversarial test Accuarcy: 62.043795620437955
Model Saved!
Epoch 00046: adjusting learning rate of group <generator object Module.parameters at 0x00000224F86A7680> to 1.0000e-04.



Total adversarial train accuarcy: 50.633797885005954
Total adversarial train loss: 22.870420150458813

[ Test epoch: 47 ]



Total benign test accuarcy: 52.89163391353173
Total adversarial test Accuarcy: 60.86468276249298
Model Saved!
Epoch 00047: adjusting learning rate of group <generator object Module.parameters at 0x00000224F86A7680> to 1.0000e-04.


0.04115182161331177:  25% 113/447 [01:16<03:39,  1.52it/s] 